# Network Analysis for Data Scientists
  
  
Bill Shi  
bill10@email.unc.edu  
Data Matters 2019

## Getting Ready

* Download and install R (https://cran.r-project.org/mirrors.html)
* Download and install RStudio Desktop (free) (https://www.rstudio.com/products/rstudio/download/)
* Open RStudio and install igraph https://igraph.org/r/
* Download and install Gephi (https://gephi.org/)

## Introduction

### Why Networks

We live in a connected world. 

* Technical infrastructure: water supply, power grids, Internet, etc.
<img src='Images/US.png'>

* Social networks: families and friends, Facebook, Twitter, etc.
<img src='Images/social_networks.png'>

* Economic networks: financial transacitons, corporate partnerships, international trades, etc.
<img src='Images/trades.png'>
Lee, et al. (2015). Journal of Sustainable Metallurgy. 1. 10.1007/s40831-015-0007-7. 

* Biological networks: neuronal networks, gene networks, food web, etc.
<img src='Images/brain.png' width="800">

* Cultural networks: semantic networks, language/literature/art connected by their similarities, etc. 
<img src='Images/wordnet.png'>

Networks describe how things are connected with each other.

### What Network Analysis can Do

* Identify important parts or connections in a network: Pagerank  
<img src='Images/google.png'>

* Predict interactions: recommender systems
<img src='Images/recommender.png'>

* Reveal sub-structure or hierarchy: community detection  
<img src='Images/blogs.png'>

* Path finding: shortest path from one node to another
<img src='Images/map.png'>

### Resources

* "Awesome Network Analysis" (https://github.com/briatte/awesome-network-analysis)
* NetSci Edu (https://sites.google.com/a/binghamton.edu/netscied/home)

<table>
    <tr>
        <td><img src='Images/newman.jpg'></td>
        <td><img src='Images/barabasi.jpg'></td>
        <td><img src='Images/statistical.jpg'></td>
    </tr>
</table>

### OUTLINE

1. Network literacy
    1. Essential Concepts and Core Ideas 
    2. Network data collection and storage
    2. Visualization and Descriptives 
3. Centrality Measures: Ranking Objects 
4. Community Detection: Clustering Objects 
5. Link Prediction: Recommendations
6. Network Models

### Format

* 40-minute lecture
* Guided practices
* 5 - 10 minute excises
* **Questions anytime**

## Link Prediction

Given an observed network, link prediction aim to solve two practical problems: find missing links or predict potential links.

### Heuristic Methods

* Common neighbors: number of common neighbors between two nodes
* Jaccard similarity: common neighbors normalized by the size of the joint neighborhood
$$ Jaccard(i,j)=\frac{|\Gamma(i)\cap\Gamma(j)|}{|\Gamma(i)\cup\Gamma(j)|} $$
* Adamic/Adar: number of weighted common neighbors; each neighbors is discounted by its degree.
$$ AdamicAdar(i,j)=\sum_{z\in\Gamma(i)\cap\Gamma(j)}\frac{1}{\log|\Gamma(z)|} $$.
* Distance: length of shortest paths.
* Kats similarity: number of paths between two nodes but the contribution from long paths is discounted.
$$ Katz(i,j)=\sum_{l=1}^{\infty}\beta^l~|paths_{i,j}^l|~~~~or~~~~Katz=(I-\beta A)^{-1}-I$$.
* Rooted Pagerank: the similarty between nodes $i$ and $j$ is the stationary probability of visiting $j$ of a random walk that starts from $i$ and returns to $i$ with probability $1-\beta$ in every step.
$$ Pagerank=(1-\beta)(I-\beta D^{-1}A)^{-1} $$

### Collaborative Filtering

Collaborative filtering is mostly used to predict edges in bi-partite networks, and is a popular method in recommendation systems. There are normally two types of nodes - users and items, and edges only exist between users and items representing, for example, which movie a user watched, what music a user like, what book a user read, and so on. 

<img src='Images/collaborativefiltering.png'>

The idea behind collaborative filtering is that individuals’ preferences can be predicted from similar others’ given large-scale population data. For concreteness, let’s consider consumers and products in e-commerce. The idea behind collaborative filtering is the following: we can probably make a good guess on what a consumer will buy next by considering other consumers that have similar purchase histories as this target consumer; on the other hand, for a consumer who has bought product A, I should probably recommend her product B, the product whose buyers have the largest overlap with A’s (e.g., the "people who bought ... also bought ..." section seen on most e-commerce platforms).

A widely used collaborative filtering technique is the latent factor model. It assumes that each individual is associated with a few latent factors, and so is each product; in other words, the individuals and the products are mapped to a same latent space. And then the (valued) interactions between the individuals and the products - e.g., ratings, likes and dislikes, and purchases - are determined by the distances between the individuals and the products in the latent space. However, the latent factors and the positions are unknown and they are usually estimated by the matrix factorization method.

Specifically, assume that each user $i$ is associated with a latent vector $P_i=(P_{i1},P_{i2},\cdots,P_{iK})$ and each item $j$ with a vector $Q_j=(Q_{j1},Q_{j2},\cdots,Q_{jK})$. The weighted edge $A_{ij}$ between $i$ and $j$ is assumed to be approximately $P_i\cdot Q_j$. Now given the observed $A_{ij}$, the goal is to find the latent vectors that minimize the following cost function:
$$
\sum_{(i,j)\in E} (A_{ij} - \sum_{k=1}^K P_{ik}Q_{jk})^2  + \beta(||P||^2 + ||Q||^2),
$$
where $E$ is the set of $(i,j)$ pairs whose $A_{ij}$ are known. This estimation is approximately a factorization of $A$, that is, $A\approx P Q^T$.

### Example: Ratings on Video Games

The data is about the world's most popular PC Gaming hub, Steam, which covers over 6,000 video games and a community of millions of users or gamers. The data is organized as a table with 3 columns (variables): user ID, game ID, and rating; and each row (observation) represent the rating from the user on the game.  

In total the data has 43,726 observations from 8,203 users and 284 games. The numbers are smaller than the size of the Steam community because we only include games that have been played by at least 50 users and users who have played at least two hours.

First, load data into R

In [ ]:
dat = read.csv('dataset-steam-2017.csv')

To obtain a fair evaluation of the method, split the data randomly into a training subset and a test subset:

In [ ]:
index = sample(seq_len(nrow(dat)), size = floor(0.9*nrow(dat)))	
train = dat[index,]
test = dat[-index,]

The collaborative filtering method is implemented by the package "recosystem", so install it before it can loaded:

In [ ]:
library('recosystem')

Next, we need to turn the training set into the format required by the recosystem package:

In [ ]:
rec_train = data_memory(user_index = train$user_id, 
                        item_index = train$item_id, 
                        rating=train$rating,
                        index1 = TRUE) 

After that we can fit the latent factor model to the training set:

In [ ]:
rec = Reco()
res = rec$train(rec_train, opts = list(ndim=10, verbose=TRUE))

where "ndim"" in the second line specifies the number of latent factors, which is set to 10 in this example.

Finally, we assess the predictive performance of the method on the test set. But first we still need to convert the test set to the format required by the package:

In [ ]:
rec_test = data_memory(user_index = test$user_id, 
                      item_index = test$item_id,
                      index1 = TRUE)

Then, the mean squared error of the predictions of ratings can be calculated as:

In [ ]:
pred = rec$predict(rec_test, out_pred = out_memory())
sum((test$rating-pred)^2) / length(pred)

### Exponential Random Graph Model

The expoential random graph model is for general link predictions, especially when nodes have attributes. It is a parametric model for the probability distribution of networks. It can be viewed as a generalization of the Logistic regression to predict edges: for every pair of nodes, the outcome is whether or not there is an edge between the two nodes and predictors are any attributes of the nodes. However, ERGM is more general and models the probability of the network as a whole instead of modelling every pair of nodes. Consequently, it even allows using any statistics of a network including global properties such as edge density and global transitivity. Most importantly, by modelling the probability of a whole network ERGM considers the correlation structure between pairs, which is overlooked in the Logistic regression model where the pairs are assumed to be independent. 

Let $A$ be a random adjacency matrix and $a$ be a realization of $A$. Assume $x(a)=(x_1,…,x_p)$ to be the vector of statistics of the network $a$ that we are interested in. ERGM specifies a distribution on $A$ as the following:
$$
P(A=a)=\frac{\exp(c \cdot x(a))}{k(c)},
$$
where $c$ is a vector of coefficients corresponding the statistics $x(a)$.

Now given an observed network and its adjacency matrix $a$, our goal is to find the coefficients $c$ that maximize the probability of observing $a$ based on the model. However, most of the time we only observe one or two snapshots of the network under study; how can we estimate this model with only one data point? This issue is addressed by advanced MCMC techniques, but the estimation process is quite complicated and is hence beyond the scope of this course. The rough idea behind it is similar to the maximal likelihood estimation for logistic regressions. Finally, the estimates of the coefficients and their statistical significance tell us how much each statistic in the vector $x(a)$ contributes to the formation of edges and the network. 

The ERGM model can be written as
$$
logit(A_{ij}=1|a_{ij}^c) = c \cdot(x(a_{ij}^+) - x(a_{ij}^- )).
$$
where logit is the logit function, $a_{ij}^c$ means all the pairs of nodes except for the $(i,j)$ pair, and $a_{ij}^+$ and $a_{ij}^-$ are the same as $a$ but with the entry $a_{ij}$ set to 1 and 0, respectively. Hence, this formula can be used to predict the edge $A_{ij}$.

### Example: UK Faculty Network

The data is about a social network consisting of 81 nodes with each representing a faculty member of a UK university, and the edges of the network represent friendships between the faculty members. There are 817 undirected, unweighted edges in total. Here we are interested in predicting edges based on the department affiliations of the faculty members.

First, load the network data into R

In [ ]:
nodes = read.csv('dataset-ukfaculty-2008-nodes.csv') 
edges = read.csv('dataset-ukfaculty-2008-edges.csv')

The ERGM is included in the "statnet" package. We need to load the  package in order to use it. If you don’t have it installed, install it first.

In [ ]:
library('statnet')

Next, we can turn the node and edge tables into a network object by the following command:

In [ ]:
colnames(edges)=c('head','tail')
G = network(edges, directed = F, matrix.type='edgelist')

Then we need to set the attributes of the nodes, which can done as the following:

In [ ]:
G %v% 'dept' = nodes$dept

Here we have two attributes for the nodes, which are the affiliations (the dept column) and the degrees (the degree column) of the nodes.

Finally, we fit the model to the network with two network statistics: the number of edges and the number of edges between nodes from the same department: 

In [ ]:
model = ergm(G ~ edges + nodematch('dept'))

The estimation results can be checked as with a regression model:

In [ ]:
summary(model)

### Excercise: How to predict? (Hint: use simulate)

In [ ]:
Aij=matrix(0,81, 81)
for (i in 1:100) {
  a = simulate.ergm(model)
  Aij = Aij + as.matrix(a)
}

Aij/100